In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

W0227 11:49:04.351834 4663846336 deprecation.py:323] From <ipython-input-1-8bf8ae5a5303>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0227 11:49:04.352683 4663846336 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0227 11:49:04.353889 4663846336 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0227 11:49:04.577899 4663846336 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import os

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Reshape, Embedding, Concatenate
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils, plot_model
import keras.backend as K
from keras.datasets import mnist

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image

import numpy as np
np.random.seed(1)

import random
import math

Using TensorFlow backend.


In [3]:
(X_train, y_train), (_, _) = mnist.load_data()

X_train = X_train.reshape(-1, 28*28)/255.

y_train = np_utils.to_categorical(y_train, num_classes=10)
print(y_train[0])
print(y_train.shape)

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
(60000, 10)


In [4]:
X_dim = X_train.shape[1] # 784
y_dim = y_train.shape[1] # 10
z_dim = 100
h_dim = 128
batch_size = 32

x = Input(shape=(X_dim,))
D_h = Dense(h_dim)(x)
D_h = LeakyReLU(0.2)(D_h)
D_o_gan = Dense(1, activation='sigmoid')(D_h)
D_o_aux = Dense(y_dim, activation='softmax')(D_h)
DM = Model(x, [D_o_gan, D_o_aux])

z = Input(shape=(z_dim,))
c = Input(shape=(y_dim,))
G_h = Concatenate(axis=1)([z, c])
G_h = Dense(h_dim)(G_h)
G_h = LeakyReLU(0.2)(G_h)
G_o = Dense(X_dim, activation='sigmoid')(G_h)
GM = Model([z, c], G_o)

# del DM
# DM = load_model('DM.h5')
# del GM
# GM = load_model('GM.h5')

DM.trainable = True
DM.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer=Adam(), metrics=['accuracy'])

g = GM([z, c])
DM.trainable = False
y, a = DM(g)
AM = Model([z, c], [y, a])
AM.compile(loss=['binary_crossentropy', 'categorical_crossentropy'], optimizer=Adam(), metrics=['accuracy'])

GM.summary()
DM.summary()
AM.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 110)          0           input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 128)          14208       concatenate_1[0][0]              
__________

/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [7]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m,n])

if not os.path.exists('out/'):
    os.makedirs('out/')

real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))
    
i=0
for it in range(1000000):
    r = np.random.randint(0, X_train.shape[0], batch_size)
    X_real, y_real = X_train[r], y_train[r]
    
    z_samples = sample_Z(batch_size, z_dim)
    x_fake = GM.predict([z_samples, y_real])
    
    Q1 = DM.train_on_batch(X_real, [real, y_real])
    Q2  = DM.train_on_batch(x_fake, [fake, y_real])
    
    D_loss_real, D_acc_real, C_loss_real, C_acc_real, _ = Q1
    D_loss_fake, D_acc_fake, C_loss_fake, C_acc_fake, _  = Q2
    
    D_loss = 0.5*(D_loss_real + D_loss_fake)
    D_acc = 0.5*(D_acc_real + D_acc_fake)
    
    C_loss = 0.5*(C_loss_real + C_loss_fake)
    C_acc = 0.5*(C_acc_real + C_acc_fake)
    
    z_samples = sample_Z(batch_size, z_dim)
    
    G_loss, G_acc, C_loss_G, C_acc_G, _ = AM.train_on_batch([z_samples, y_real], [real, y_real])
    
    if it%1000==0:
        print(Q1)
        print(Q2)
        DM.save('DM.h5')
        GM.save('GM.h5')
        
        for j in range(y_dim):
            y_samples = np.ones((16,1))*j
            y_samples = np_utils.to_categorical(y_samples, num_classes=y_dim)
            samples = GM.predict([sample_Z(16, z_dim), y_samples])
            fig = plot(samples)
            plt.savefig('out/{}.png'.format(str(i).zfill(3) + '_' + str(j)), bbox_inches='tight')
            plt.close(fig)
        i+=1
        
        print('iter:{}'.format(it))
        print('[D loss:{:.4}, acc:{:.4}] [G loss:{:.4}, acc:{:.4}]'.format(D_loss, D_acc, G_loss, G_acc))
        print()

[2.6421041, 0.4971919, 2.1449122, 0.875, 0.125]
[2.4833164, 0.025043493, 2.458273, 1.0, 0.09375]
iter:0
[D loss:2.563, acc:0.2611] [G loss:6.683, acc:4.38]

[0.18355463, 0.016265348, 0.16728929, 1.0, 0.9375]
[1.031906, 0.00057849666, 1.0313275, 1.0, 0.6875]
iter:1000
[D loss:0.6077, acc:0.008422] [G loss:9.228, acc:8.189]

[0.3854921, 0.039306626, 0.34618545, 0.96875, 0.90625]
[0.11403547, 0.011400534, 0.10263494, 1.0, 1.0]
iter:2000
[D loss:0.2498, acc:0.02535] [G loss:6.155, acc:6.06]

[0.1848998, 0.020779036, 0.16412078, 1.0, 0.96875]
[0.19850904, 0.016787164, 0.18172187, 1.0, 0.9375]
iter:3000
[D loss:0.1917, acc:0.01878] [G loss:5.163, acc:5.012]

[0.18854474, 0.018848564, 0.16969617, 1.0, 0.96875]
[0.18317136, 0.12568867, 0.057482686, 0.96875, 1.0]
iter:4000
[D loss:0.1859, acc:0.07227] [G loss:3.251, acc:3.199]

[0.41019076, 0.20905977, 0.20113099, 0.9375, 0.9375]
[0.09314371, 0.06340798, 0.02973573, 1.0, 1.0]
iter:5000
[D loss:0.2517, acc:0.1362] [G loss:3.831, acc:3.799]

[0.3

KeyboardInterrupt: 